#### import modules

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-muted')
from matplotlib import cm
import geopandas as gpd

from pyproj import Transformer
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import multiprocessing
from multiprocessing import Pool
from functools import partial
import numpy.random as npr
import matplotlib.dates as mdates
from itertools import product
import copy
import pickle
from tqdm import tqdm
import boto3
import io
import yaml

import nomad.io.base as loader
import nomad.city_gen as cg
from nomad.city_gen import City, Building
import nomad.traj_gen as tg
from nomad.traj_gen import Agent, Population
import nomad.stop_detection.ta_dbscan as DBSCAN
import nomad.stop_detection.lachesis as Lachesis
from nomad.generation.sparsity import gen_params_target_q, gen_params_ranges

#### load in config file

In [14]:
config_path = 'synthetic-data-config.yaml'

with open (config_path, 'r') as f:
    config = yaml.safe_load (f)

# randomization seed
seed = config['randomization']['seed']

# Garden City info
map_geojson = config['map']['read']
map_pkl = config['map']['load']

# trajectory info
generate_full_traj = config['trajectory']['full']
generate_sparse_traj = config['trajectory']['sparse']
generate_diaries = config['trajectory']['diaries']

# agent info
N = config['agents']['n']
name_count = config['agents']['name_count']

#### load in the city

In [9]:
city_geojson = gpd.read_file(map_geojson)
city = cg.load(map_pkl)

#### functions used to map Garden City's coordinates onto a location in the Atlantic Ocean

In [12]:
def garden_city_to_lat_long(agent, sparse_traj=True, full_traj=False, diaries=True):
    def project_city_blocks_to_web_mercator(df):
        """Convert (x, y) from 15m block units to Web Mercator meters via affine shift and projection."""
        transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
        df['x'] = 15 * df['x'] - 4265699
        df['y'] = 15 * df['y'] + 4392976
        if 'ha' in df:
            df['ha'] = 15 * df['ha']
        df['longitude'], df['latitude'] = transformer.transform(df['x'].values, df['y'].values)
        df['date'] = df['datetime'].dt.date
        return df

    def finalize(df):
        front = ['identifier', 'timestamp', 'longitude', 'latitude', 'x', 'y', 'date']
        cols = [col for col in front if col in df] + [col for col in df.columns if col not in front]
        return df[cols].rename(columns={'identifier': 'uid', 'timestamp': 'timestamp'}).reset_index(drop=True)

    if sparse_traj:
        agent.sparse_traj = finalize(project_city_blocks_to_web_mercator(agent.sparse_traj))
    if full_traj:
        agent.trajectory = finalize(project_city_blocks_to_web_mercator(agent.trajectory))
        
    if diaries:
        diary = agent.diary.copy()
        xs = []
        ys = []
        for loc in diary["location"]:
            if loc is None:
                xs.append(None)
                ys.append(None)
            else:
                pt = agent.city.buildings[loc].geometry.centroid
                xs.append(pt.x)
                ys.append(pt.y)
        diary["x"] = xs
        diary["y"] = ys
        agent.diary = finalize(project_city_blocks_to_web_mercator(diary))

In [13]:
def philly_to_lat_long(agent, sparse_traj=True, full_traj=False, diaries=True):
    def project_point_to_web_mercator(x, y):
        """
        Project a fractional (x, y) block coord to Web Mercator using affine interpolation.
        philly_grid_map is the grid_map produced by RealCityGenerator in virtual_philly.ipynb
           import it into this notebook through pkl
        """
        i, j = int(math.floor(x)), int(math.floor(y))
        poly = philly_grid_map.get((i, j))

        if poly is None:
            raise ValueError(f"No polygon found at grid cell ({i}, {j})")

        # Bounds of the 1x1 block polygon in EPSG:3857
        minx, miny, maxx, maxy = poly.bounds

        dx = x - i
        dy = y - j

        X = minx + dx * (maxx - minx)
        Y = miny + dy * (maxy - miny)

        return X, Y

    def apply_projection_to_df(df):
        """Apply Web Mercator projection to a DataFrame with 'x' and 'y' columns."""
        def safe_project(row):
            try:
                return project_point_to_web_mercator(row['x'], row['y'])
            except Exception:
                return (None, None)

        projected = df.apply(safe_project, axis=1)
        df[['x', 'y']] = pd.DataFrame(projected.tolist(), index=df.index)

        transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
        if 'ha' in df:
            df['ha'] = 10 * df['ha']  # 10 because thats the sidelength of a block
        df['longitude'], df['latitude'] = transformer.transform(df['x'].values, df['y'].values)
        df['date'] = df['datetime'].dt.date
        return df

    def finalize(df):
        front = ['identifier', 'timestamp', 'longitude', 'latitude', 'x', 'y', 'date']
        cols = [col for col in front if col in df] + [col for col in df.columns if col not in front]
        return df[cols].rename(columns={'identifier': 'uid', 'timestamp': 'timestamp'}).reset_index(drop=True)

    if sparse_traj:
        agent.sparse_traj = finalize(apply_projection_to_df(agent.sparse_traj))
    if full_traj:
        agent.trajectory = finalize(apply_projection_to_df(agent.trajectory))

    if diaries:
        diary = agent.diary.copy()
        xs = []
        ys = []
        for loc in diary["location"]:
            if loc is None:
                xs.append(None)
                ys.append(None)
            else:
                pt = agent.city.buildings[loc].geometry.centroid
                xs.append(pt.x)
                ys.append(pt.y)
        diary["x"] = xs
        diary["y"] = ys
        agent.diary = finalize(apply_projection_to_df(diary))